**EXPLORACIÓN DE CONTENIDOS MULTIMEDIA BASADA EN LA SIMILITUD**     

**SIMILARITY-BASED BROWSING OF MULTIMEDIA CONTENTS**




# Primeros pasos: inicialización del entorno y carga del conjunto de datos a explorar

In [ ]:
#@title <font size = 5> Importar e instalar dependencias. { display-mode: "form" }
!pip install torch
!pip install sentence_transformers
!pip install mysql
!pip install mysql-connector
!pip install pyyaml==5.4.1
!pip install pymysql
!pip install faiss-gpu
!pip install hdbscan
!pip install umap-learn

import umap
import hdbscan
import textwrap
import os
import torch
import pandas as pd
import sys
import pickle
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import mysql
import mysql.connector
import pymysql
import faiss
import matplotlib.pyplot as plt
import plotly.express as px

from IPython.display import clear_output 
from sentence_transformers import CrossEncoder
from sklearn.decomposition import PCA
from torch.nn.functional import normalize
from sentence_transformers.cross_encoder import CrossEncoder
from psutil import virtual_memory
from google.colab import drive
from sentence_transformers import SentenceTransformer, util, models
from datetime import datetime
from google.colab import drive
from sklearn.decomposition import PCA
from mysql.connector import Error
from sqlalchemy import create_engine
from IPython.display import HTML, display
from google.colab import data_table
from google.colab import output

print("\nConectando con Google Drive...\n")
drive.mount('/content/drive')

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

data_table._DEFAULT_FORMATTERS[float] = lambda x: f"{x:.2f}"

clear_output()

In [ ]:
#@title <font size = 5> Comprobar la GPU y RAM disponible. { display-mode: "form" }

#@markdown Esta celda comprueba si Google Colab le ha asignado una GPU o no y la cantidad de RAM que le ha concedido.

#@markdown No es necesario disponer de una GPU, pero el uso de la misma hará que el código se ejecute mucho más rápido.

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU\n\n')
else:
  print(gpu_info, '\n\n')

ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM.\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime.')
else:
  print('You are using a high-RAM runtime.')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
#@title Iniciar el directorio de trabajo y cargar datos. { display-mode: "form" }

#@markdown Para ejecutar el algoritmo conectaremos el cuaderno con Google Drive para almacenar la información.Esta celda conecta su cuenta drive con este notebook de Google Colab. Además inicializará el workspace necesario para ejecutar el código, creará 3 directorios: **similarity_matrix**, **saved_embeddings** y **saved_models** para almacenar las similitudes de los elementos, sus embeddings y los modelos que se deseen almacenar respectivamente.

#@markdown Para ello se deben indicar los siguientes argumentos:

#@markdown *   **WS_PATH**: Ruta del directorio de trabajo.Es la carpeta en la que se crearán los directorios anteriormente mencionados. **(Ejemplo:᠎ ᠎/content/drive/general_explorer/)**.

#@markdown *   **FILE_NAME**: Nombre del conjunto de datos que deseas explorar. Debe estar en la carpeta WS_PATH y debe de ser un fichero de  formato csv. **(Ejemplo: arxiv_data.csv)**.

#@markdown *   **TITLE_COLUM**: Nombre de la columna del conjunto de datos que contiene los títulos de los elementos.

#@markdown *   **EMBD_COLUM**: Nombre de la columna del conjunto de datos que contiene los textos sobre los que se realizará la exploración.

#@markdown *   **LOAD_FULL_CSV**: Si la casilla está marcada, se cargará el fichero indicado en FILE_NAME entero, en otro caso se deberá indicar el número de filas que desea cargar en el siguiente argumento.

#@markdown *   **NUMBER_OF_ROWS**: Número de filas que serán cargadas del archivo indicado en FILE_NAME si no está marcada la anterior casilla.

#@markdown  <br>

WS_PATH ="/content/drive/" #@param {type:"string"}
FILE_NAME = "arxiv_data.csv" #@param {type:"string"}
EMBD_COLUMN = "abstracts" #@param {type:"string"}
TITLE_COLUMN = "titles" #@param {type:"string"}
LOAD_FULL_CSV = True #@param {type:"boolean"}
NUMBER_OF_ROWS =  100#@param {type:"integer"}

print("\nInicializando...\n")

def ensure_mkdir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

similarity_matrix_dir = WS_PATH + "similarity_matrix/"
saved_embeddings_dir = WS_PATH + "saved_embeddings/"
saved_models_dir = WS_PATH + "saved_models/"

ensure_mkdir(similarity_matrix_dir)
ensure_mkdir(saved_embeddings_dir)
ensure_mkdir(saved_models_dir)

if LOAD_FULL_CSV:
  df = pd.read_csv(WS_PATH + FILE_NAME)
else:
  df = pd.read_csv(WS_PATH + FILE_NAME,nrows=NUMBER_OF_ROWS)

print("Columnas cargadas: ", list(df.columns))
if EMBD_COLUMN not in df.columns:
  raise Exception("La columna {} no se encuentra en el conjunto de datos cargado.".format(EMBD_COLUMN))


#df['id'] = range(0, len(df))
print("Número de elementos cargados: ", len(df), '\n')
print("Mostrando los primeros elementos:\n")

data_table.DataTable(df.head(), include_index=True, num_rows_per_page=5)

# Cálculo o carga de los sentence embeddings y exploración en el espacio: Bi-Encoder.

En esta sección se calculan o cargan los sentence embeddings de los textos indicados del conjunto de datos.

Para crear los embeddings existe una larga lista de modelos ya entrenados que se encuentran disponibles en [HuggingFace](https://huggingface.co/), se puede observar el rendimiento de los mismos [aquí](https://www.sbert.net/_static/html/models_en_sentence_embeddings.html).

In [ ]:
#@title Iniciar el modelo: { vertical-output: true, display-mode: "form" }

#@markdown En esta celda se inicializa el modelo y se puede observar gráficamente la distribución de las longitudes de los textos.

#@markdown Se deben indicar los siguientes parámetros:

#@markdown *   **MODEL_NAME_BIENCODER**: Nombre del modelo que se va a utilizar para generar los sentence embeddings. Se puede seleccionar cualquiera de la lista desplegable o de la web HugginFace. Si el modelo esta en la carpeta de saved_models se cargará de esa carpeta.

#@markdown *   **SAVE_MODEL**: Si esta marcado se guarda el modelo en la carpeta de saved_models. Así, al cargar el modelo, primero mira si esta guardado para cargarlo de esta carpeta.

#@markdown <br>

MODEL_NAME_BIENCODER = "sentence-transformers/all-MiniLM-L6-v2" #@param ["sentence-transformers/paraphrase-MiniLM-L6-v2", "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", "sentence-transformers/LaBSE", "sentence-transformers/all-mpnet-base-v2", "sentence-transformers/all-distilroberta-v1", "sentence-transformers/multi-qa-mpnet-base-dot-v1", "sentence-transformers/all-MiniLM-L6-v2", "stsb-roberta-large"] {allow-input: true}
SAVE_MODEL = False #@param {type:"boolean"}

print("Initializing the model...\n")
if os.path.exists(WS_PATH + "saved_models/" + MODEL_NAME_BIENCODER):
  BIENCODER_MODEL = SentenceTransformer(model_name_or_path = WS_PATH + "saved_models/" + MODEL_NAME_BIENCODER)
else:
  BIENCODER_MODEL = SentenceTransformer(model_name_or_path = MODEL_NAME_BIENCODER)

print('Modelo descargado:\n')
print(BIENCODER_MODEL,'\n')

df[EMBD_COLUMN] = df[EMBD_COLUMN].astype("string")
try:
  df[EMBD_COLUMN].str.split().\
    map(lambda x: len(x)).plot.hist(bins=200,alpha = 0.7,figsize=(15,5),title = 'Frecuencia del número de palabras por sentencia.',legend=True)
except:
  print("No se puede dibujar la gráfica porque no todos los elementos son strings.")

#Movemos modelo a GPU
print('Moving model to', device, ".")
BIENCODER_MODEL = BIENCODER_MODEL.to(device)

if SAVE_MODEL:
  BIENCODER_MODEL.save(WS_PATH + "saved_models/" + MODEL_NAME_BIENCODER)

In [ ]:
#@title Calcular los sentence embeddings. { display-mode: "form" }

#@markdown En la celda actual se deben indicar los siguientes parámetros:

#@markdown *   **MAX_SEQ_LEN** : Maxima longitud de los textos a procesar.Textos con un número mayor de palabras serán truncados. Si es -1 se usará el valor por defecto del modelo seleccionado. Puedes ayudarte de la gráfica anteriormente generada para ajustar este valor. **(Ejemplo: 512)**.

#@markdown *   **NORMALIZE_EMBEDDINGS** :  Si está marcada los embeddings serán normalizados. Recomendable si se quiere obtener similitudes entre 0 y 1.

#@markdown *   **SAVE_EMBEDDINGS** : Si está marcada los embeddings serán guardados con el nombre indicado en el parametro OUTPUT_NAME_EMBEDDINGS.

#@markdown *   **OUTPUT_NAME_EMBEDDINGS** : Nombre con el que serán guardados los embeddings. **(Ejemplo: embeddings-peliculas)**

BIENCODER_BATCH_SIZE = 32
MAX_SEQ_LEN =  -1#@param {type:"integer"}
NORMALIZE_EMBEDDINGS = True #@param {type:"boolean"}
SAVE_EMBEDDINGS = True #@param {type:"boolean"}

OUTPUT_NAME_EMBEDDINGS = "embeddings-abstracts-arxiv" #@param {type:"string"}

if MAX_SEQ_LEN != -1:
  BIENCODER_MODEL._first_module().max_seq_length  = MAX_SEQ_LEN

print('Usando el modelo:\n')
print(BIENCODER_MODEL)

print('\nComputing embeddings...')
sentences = [str(x) for x in df[EMBD_COLUMN].tolist()]
embeddings = BIENCODER_MODEL.encode(sentences,
                                    convert_to_tensor=False,
                                    device = str(device),
                                    batch_size = BIENCODER_BATCH_SIZE,
                                    show_progress_bar=True,
                                    normalize_embeddings= NORMALIZE_EMBEDDINGS)

def is_normalized(sentence_embeddigs):
  for e in sentence_embeddigs:
    norma =np.sqrt(sum(e**2))
    if norma < 0.99 or norma > 1.01:
      return False
  return True

if not(is_normalized(embeddings)):
  print("\nAviso! Los vectores no estan normalizados en la salida de este modelo. Se recomienda marcar la casilla 'NORMALIZE_EMBEDDINGS' para que\nlas similitudes se encuentren en un rango entre 0 y 1 , para así poder interpretarlas de manera más sencilla.")

if SAVE_EMBEDDINGS:
  print('Guardando embeddings...')
  with open(saved_embeddings_dir + OUTPUT_NAME_EMBEDDINGS +'.pkl', "wb") as fOut:
    pickle.dump(embeddings, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#@title Cargar embeddings. { display-mode: "form" }

EMBEDDINGS_FILE_NAME = "embeddings-abstracts-arxiv" #@param {type:"string"}

try:
  with open(saved_embeddings_dir + EMBEDDINGS_FILE_NAME +'.pkl', "rb") as fIn:
      embeddings = pickle.load(fIn)
except Exception as e:
  raise
else:
  print("Embeddings cargados correctamente.")

In [ ]:
#@title Explorar embeddings en el espacio. { display-mode: "form" }


# embeddings = embeddings_aux.cpu()

n_neighbors= 25#valores pequeños mantienen mas informacion local,  valores grandes mas informacion global
umap_embeddings = umap.UMAP(n_neighbors=n_neighbors, 
                            n_components=10, 
                            metric='cosine').fit_transform(embeddings)
clusters = hdbscan.HDBSCAN(min_cluster_size=3,
                          metric='euclidean',                      
                          cluster_selection_method='eom').fit(umap_embeddings)
pca = PCA(n_components=3)
pca.fit(umap_embeddings) 
X_pca = pca.transform(umap_embeddings) 

Xax = X_pca[:,0]
Yax = X_pca[:,1]
Zax = X_pca[:,2]

try:
  fig = px.scatter_3d(df,
                      x=Xax,
                      y=Yax,
                      z=Zax,
                      hover_data =[TITLE_COLUMN],
                      hover_name = df.index,
                      height=1000,
                      opacity=0.7,
                      color = clusters.labels_)
except:
  print("Columna TITLE_COLUMN no encontrada, solo mostrando index el en cada punto")
  try:
    fig = px.scatter_3d(df,
                        x=Xax,
                        y=Yax,
                        z=Zax,
                        hover_name = df.index,
                        height=1000,
                        opacity=0.7,
                        color = clusters.labels_)
  except:
    raise
  
fig.show()

#Exploración en tiempo real.

En esta sección se puede explorar el conjunto de datos en tiempo real sin necesidad de tener calculada la matriz de similitud. Es necesario haber ejecutado la creación o el cargado de los sentence embeddings antes de ejecutar a la exploración.

In [ ]:
#@title Explorar uno con todos. { display-mode: "form" }

#@markdown *   **INDEX_IN_CSV** : Indica el index del elemento del que se desea obtener sus similares.

#@markdown *   **NUM_SAVE** : Número de elementos similares que se desea obtener.

#@markdown <br>

INDEX_IN_CSV = 55 #@param {type:"integer"}
NUM_SAVE =  20#@param {type:"integer"}

#Configuramos faiss
if torch.cuda.is_available():
  cfg = faiss.GpuIndexFlatConfig()
  cfg.useFloat16 = True
  index = faiss.GpuIndexFlatIP(faiss.StandardGpuResources(), embeddings.shape[1], cfg)
else:
  index = faiss.IndexFlatIP(embeddings.shape[1])

index.add(embeddings)

if NUM_SAVE < 0:
  NUM_SAVE = 0
if NUM_SAVE >= len(df):
  NUM_SAVE = len(df)  - 1
  
cosenos, indices = index.search(np.array([embeddings[INDEX_IN_CSV]]), NUM_SAVE + 1)
cosenos = cosenos[0]
indices = indices[0]


print("Texto a explorar:\n")
print(textwrap.fill(df[EMBD_COLUMN][INDEX_IN_CSV][0:465], 70))

if TITLE_COLUMN == "":
  df_similares = pd.DataFrame(df.iloc[indices[1:]], columns = {EMBD_COLUMN, 'similarity'})
  df_similares = df_similares[['similarity', EMBD_COLUMN]]
else:
  df_similares = pd.DataFrame(df.iloc[indices[1:]], columns = {TITLE_COLUMN, EMBD_COLUMN, 'similarity'})
  df_similares = df_similares[['similarity', TITLE_COLUMN, EMBD_COLUMN]]

df_similares['similarity'] = cosenos[1:]

data_table.DataTable(df_similares, include_index=False, num_rows_per_page=5)

In [ ]:
#@title Explorar todos con tu propio texto. { display-mode: "form" }

#@markdown *   **TEXT** : Texto con el que se desea comparar los elementos del conjunto de datos.

#@markdown *   **NUM_SAVE** : Número de elementos similares que se desean obtener.

#@markdown

TEXT =  "deep neural network for games" #@param {type:"string"}
NUM_SAVE =  15#@param {type:"integer"}

query_embedding = BIENCODER_MODEL.encode([TEXT],
                                    convert_to_tensor=False,
                                    device = str(device),
                                    show_progress_bar=True,
                                    normalize_embeddings= True)

clear_output()

#Configuramos faiss
if torch.cuda.is_available():
  cfg = faiss.GpuIndexFlatConfig()
  cfg.useFloat16 = True
  index = faiss.GpuIndexFlatIP(faiss.StandardGpuResources(), embeddings.shape[1], cfg)
else:
  index = faiss.IndexFlatIP(embeddings.shape[1])

index.add(embeddings)

if NUM_SAVE <= 0:
  NUM_SAVE = 1
if NUM_SAVE > len(df):
  NUM_SAVE = len(df)
  
cosenos, indices = index.search(np.array(query_embedding), NUM_SAVE)
cosenos = cosenos[0]
indices = indices[0]

print("Texto a explorar:\n")
print(textwrap.fill(TEXT, 230))

if TITLE_COLUMN == "":
  df_similares = pd.DataFrame(df.iloc[indices],columns = {EMBD_COLUMN, 'similarity'})
  df_similares = df_similares[['similarity', EMBD_COLUMN]]
else:
  df_similares = pd.DataFrame(df.iloc[indices],columns = {TITLE_COLUMN, EMBD_COLUMN, 'similarity'})
  df_similares = df_similares[['similarity', TITLE_COLUMN, EMBD_COLUMN]]


df_similares['similarity'] = cosenos

data_table.DataTable(df_similares, include_index=False, num_rows_per_page=5)

#Cálculo o carga de similitudes: Faiss.

En esta sección se calcula o carga la matriz de similitudes entre los sentence embeddings utilizando Faiss.

In [ ]:
#@title Cálculo de similitudes - Bi-encoder { display-mode: "form" }

#@markdown Esta celda permite calcular la matriz de similitudes. Para ello se deben de indicar los siguientes parámetros:

#@markdown *   **NUM_SAVE**: Número de elementos similares que se quieren guardar para cada uno de los elementos del conjunto de datos.

#@markdown *   **SAVE_SIMILARITY_MATRIX_PICKLE**: Si la casilla está marcada la matriz de similitudes será guardada en formato .pkl con el nombre indicado en OUTPUT_NAME. Si se quieren cargar posteriormente en este cuaderno se deben de guardar con este formato. 

#@markdown *   **SAVE_SIMILARITY_MATRIX_CSV**: Si la casilla está marcada la matriz de similitudes será guardada en formato .csv con el nombre indicado en OUTPUT_NAME. Usar cuando se quiera exportar los datos a otra plataforma en la que utilizar el archivo en formato csv.

#@markdown *   **OUTPUT_NAME**: Nombre del fichero en el que se guardará la matriz de similitudes generada.

#@markdown <br>

NUM_SAVE = 987632469 #@param {type:"integer"}
SAVE_SIMILARITY_MATRIX_PICKLE = False #@param {type:"boolean"}
SAVE_SIMILARITY_MATRIX_CSV = False #@param {type:"boolean"}
OUTPUT_NAME = "similarity_matrix-abstracts-arxiv" #@param {type:"string"}

def fun_cosenos_biencoder(NUM_SAVE: int):
 #Configuramos faiss
  if torch.cuda.is_available():
    cfg = faiss.GpuIndexFlatConfig()
    cfg.useFloat16 = True
    index = faiss.GpuIndexFlatIP(faiss.StandardGpuResources(), embeddings.shape[1], cfg)
  else:
    index = faiss.IndexFlatIP(embeddings.shape[1])
    
  index.add(embeddings)

  D, I = index.search(embeddings, NUM_SAVE + 1)
  return [lista[1:] for lista in D], [lista[1:] for lista in I]


if NUM_SAVE > len(df):
  NUM_SAVE = len(df) - 1
if NUM_SAVE > 2047:
  NUM_SAVE = 2047

cosenos_biencoder, index_biencoder = fun_cosenos_biencoder(NUM_SAVE)
df['index_biencoder'] = index_biencoder
df['cosenos_biencoder'] = cosenos_biencoder

#Al guardar listas como csv y luego leerlas se convierten a string, por lo que se dejan los valores separados por
#comas para poder parsearlo de manera más sencilla al leerlo
df['index_biencoder'] = df['index_biencoder'].map(lambda x: ','.join(map(str, x)))
df['cosenos_biencoder'] = df['cosenos_biencoder'].map(lambda x: ','.join(map(str, x)))

if SAVE_SIMILARITY_MATRIX_CSV:
  print("Guardando {}.csv...".format(OUTPUT_NAME))
  df.to_csv(similarity_matrix_dir + OUTPUT_NAME +'.csv',columns={'cosenos_biencoder','index_biencoder'},index=False)
if SAVE_SIMILARITY_MATRIX_PICKLE:
  print("Guardando {}.pkl...".format(OUTPUT_NAME))
  with open(similarity_matrix_dir + OUTPUT_NAME +'.pkl', "wb") as fOut:
    pickle.dump({'cosenos_biencoder': cosenos_biencoder, 'index_biencoder': index_biencoder}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

print("Mostrando los primeros elementos...")
data_table.DataTable(df[['index_biencoder','cosenos_biencoder']].head(), include_index=True, num_rows_per_page=5)


In [ ]:
#@title Cargar matriz de similitudes - Bi-encoder. { display-mode: "form" }
#@markdown Si se tiene la matriz de similitud ya calculada,se puede cargar el fichero con la misma y evitar tener que calcularla de nuevo. Indicar el nombre del fichero (en formato .pkl):

MATRIX_FILE_NAME = "similarity_matrix-abstracts-arxiv" #@param {type:"string"}

try:
  with open(similarity_matrix_dir + MATRIX_FILE_NAME +'.pkl', "rb") as fIn:
      resp = pickle.load(fIn)
      cosenos_biencoder = resp['cosenos_biencoder']
      index_biencoder = resp['index_biencoder']
      df['index_biencoder'] = index_biencoder
      df['cosenos_biencoder'] = cosenos_biencoder
except Exception as e:
  raise e
else:
  print("Matriz de similitudes cargada correctamente.")


#Re-ranking o carga de las similitudes: Cross-Encoder.

En esta sección se calcula o carga la matriz de similitudes entre los sentence embeddings utilizando el Cross-encoder.

In [ ]:
#@title Re-ranking de resultados : Cross-encoder. { display-mode: "form" }

#@markdown En esta sección se obtendrá un nuevo orden (re-reranking) para los resultados obtenidos con el Bi-encoder. Este tipo de modelo es mucho más lento pero más preciso, por lo que se recomienda que el número de elementos similares que se desean re-ordenar de cada elemento del conjunto de datos no sea muy grande. <b>(No se recomienda ejecutar sin disponer de una GPU)

#@markdown En la esta celda se inicializa el modelo Cross-encoder a utilizar.

#@markdown Se deben rellenar los siguientes parámetros:

#@markdown *   **MODEL_NAME_CROSSENCODER**: Nombre del modelo Cross-encoder que se va a utilizar.

#@markdown *   **NUM_SAVE**: Número de elementos similares a cada elemento obtenidos por el Bi-encoder que se desean re-rankear.

#@markdown *   **SAVE_SIMILARITY_MATRIX_PICKLE**: Si la casilla está marcada la matriz de similitudes será guardada en formato .pkl con el nombre indicado en OUTPUT_NAME. Si se quieren cargar posteriormente en este cuaderno se deben de guardar con este formato. 

#@markdown *   **SAVE_SIMILARITY_MATRIX_CSV**: Si la casilla está marcada la matriz de similitudes será guardada en formato .csv con el nombre indicado en OUTPUT_NAME. Usar cuando se quiera exportar los datos a otra plataforma en la que utilizar el archivo en formato csv.

#@markdown *   **OUTPUT_NAME**: Nombre del fichero en el que se guardará la matriz de similitudes generada.

#@markdown <br>

MODEL_NAME_BIENCODER = 'cross-encoder/ms-marco-MiniLM-L-12-v2' #@param {type:"string"}
NUM_SAVE =  10#@param {type:"integer"}
SAVE_SIMILARITY_MATRIX_PICKLE = True #@param {type:"boolean"}
SAVE_SIMILARITY_MATRIX_CSV = False #@param {type:"boolean"}
OUTPUT_NAME = "similarity_matrix-crossencoder-abstracts-arxiv" #@param {type:"string"}

model_cross_encoder = CrossEncoder(MODEL_NAME_BIENCODER, num_labels = 1)

def fun_cosenos_1_crossencoder(elemento: int, listaElementos):
  query = str(listaElementos[elemento]) #LOS NAN DAN ERROR. Añadimos str para evitarlo y funciona.
  # With all sentences in the corpus
  corpus = [str(listaElementos[ind]) for ind in index_biencoder[elemento][0:NUM_SAVE]]

  # So we create the respective sentence combinations
  sentence_combinations = [[query, corpus_sentence] for corpus_sentence in corpus]

  # Compute the similarity scores for these combinations
  cosenos_1_crossencoder = model_cross_encoder.predict(sentence_combinations, show_progress_bar = False, convert_to_tensor=True)
  D, I = torch.topk(cosenos_1_crossencoder, k = len(cosenos_1_crossencoder))

  return D.tolist(), I.tolist()

def fun_cosenos_crossencoder(elemento_ini: int, elemento_fin: int, listaElementos):

  cosenos_crossencoder = []
  index_crossencoder = []
  for i in range(elemento_ini, elemento_fin):
    out.update(progress(i, elemento_fin))
    resultado = fun_cosenos_1_crossencoder(i, listaElementos)
    cosenos_crossencoder.append(resultado[0])
    index_crossencoder.append([index_biencoder[i][x] for x in resultado[1]])

  return cosenos_crossencoder, index_crossencoder

if len(index_biencoder) > 0 and NUM_SAVE > len(index_biencoder[0]):
  NUM_SAVE = len(index_biencoder[0])

if NUM_SAVE <= 0:
  raise Exception('NUM_SAVE debe ser mayor que 0 y menor o igual al NUM_SAVE utilizado en el Biencoder.')

out = display(progress(0, len(df)), display_id=True)

cosenos_crossencoder, index_crossencoder = fun_cosenos_crossencoder(0, len(df), df[EMBD_COLUMN].tolist())
df['index_crossencoder'] = index_crossencoder
df['cosenos_crossencoder'] = cosenos_crossencoder

if SAVE_SIMILARITY_MATRIX_CSV:
  print("Guardando {}.csv...".format(OUTPUT_NAME))
  df.to_csv(similarity_matrix_dir + OUTPUT_NAME +'.csv',columns={'cosenos_crossencoder','index_crossencoder'},index=False)
if SAVE_SIMILARITY_MATRIX_PICKLE:
  print("Guardando {}.pkl...".format(OUTPUT_NAME))
  with open(similarity_matrix_dir + OUTPUT_NAME +'.pkl', "wb") as fOut:
    pickle.dump({'cosenos_crossencoder': cosenos_crossencoder, 'index_crossencoder': index_crossencoder}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#@title Cargar matriz de similitudes : Cross-encoder. { display-mode: "form" }
#@markdown Si se tiene la matriz de similitud ya calculada,se puede cargar el fichero con la misma y evitar tener que calcularla de nuevo. Indicar el nombre del fichero (en formato .pkl):

MATRIX_FILE_NAME = "similarity_matrix-crossencoder-abstracts-arxiv" #@param {type:"string"}

try:
  with open(similarity_matrix_dir + MATRIX_FILE_NAME +'.pkl', "rb") as fIn:
      resp = pickle.load(fIn)
      cosenos_crossencoder = resp['cosenos_crossencoder']
      index_crossencoder = resp['index_crossencoder']
      df['index_crossencoder'] = index_crossencoder
      df['cosenos_crossencoder'] = cosenos_crossencoder
except Exception as e:
  print("Hay algún problema con el fichero indicado.")
  raise
else:
  print("Matriz de similitudes cargada correctamente.")

#Exploración de la matriz de similitud.

En esta sección se pueden explorar los resultados obtenidos anteriormente con el Bi-encoder o el Cross-encoder. Es necesario haber ejecutado al menos el calculo de las similitudes con el Bi-encoder antes de lanzarte a la exploración.





In [ ]:
#@title Obtener similares a un elemento dado. { display-mode: "form" }

#@markdown *   BI_CO: Indica el encoder sobre el que quieres explorar los resultados.

#@markdown *   INDEX_IN_CSV: Indica el índice del elemento del que deseas explorar las similares.

BI_CO = "Bi-encoder" #@param ['Bi-encoder', 'Cross-encoder'] {allow-input: false}
INDEX_IN_CSV =  0#@param {type:"integer"}

print("Texto a explorar:\n")
print(textwrap.fill(df[EMBD_COLUMN][INDEX_IN_CSV], 230))

if BI_CO == 'Bi-encoder':
  indexes_aux = index_biencoder
  cosenos_aux = cosenos_biencoder
elif BI_CO == 'Cross-encoder':
  indexes_aux = index_crossencoder
  cosenos_aux = cosenos_crossencoder
else:
  raise Exception("El encoder proporcionado no es valido")

df_similares = pd.DataFrame(df.iloc[indexes_aux[INDEX_IN_CSV]], columns = {EMBD_COLUMN, 'similarity'})
df_similares['similarity'] = cosenos_aux[INDEX_IN_CSV]

data_table.DataTable(df_similares, include_index=True, num_rows_per_page=5)

#Exportar dataframe a una base de datos
Se exporta el conjunto de datos cargados junto a las columnas que contienen los indices y similitudes de las más parecidas a cada elemento a una base de datos MYSQL

In [ ]:
#@title  { display-mode: "form" }
#@markdown *   **MYSQL_HOST**: Dirección del host de la DB. (Ejemplo: 82.54.219.71:3306 o localhost)

#@markdown *   **MYSQL_USER**: Usuario de la DB (Ejemplo: root).

#@markdown *   **MYSQL_PASSWORD**: Contraseña del usuario de la DB. 

#@markdown *   **MYSQL_DB**: Nombre del schema de la DB.

#@markdown *   **MYSQL_TABLE_NAME**: Nombre de la tabla a la que se desea exportar los datos.

MYSQL_HOST = "localhost" #@param {type:"string"}
MYSQL_USER = "root" #@param {type:"string"}
MYSQL_PASSWORD = "localhost" #@param {type:"string"}
MYSQL_DB = "bd_tfg" #@param {type:"string"}
MYSQL_TABLE_NAME = "tfg" #@param {type:"string"}


engine = create_engine("mysql+pymysql://" + MYSQL_USER + ":" + MYSQL_PASSWORD + "@" + MYSQL_HOST + "/" + MYSQL_DB)
df.to_sql(MYSQL_TABLE_NAME, con=engine, if_exists='replace', index = False)

# Exploración con otro conjunto de datos.
En esta sección se puede explorar el conjunto de datos cargado con el que se le indica por parámetos. Es necesario haber ejecutado la inicialización del modelo y la creación o el cargado de embeddings del conjunto original antes de ejecutar a la exploración con el nuevo conjunto de datos.

In [ ]:
#@title  { display-mode: "form" }

#@markdown Se deben indicar los siguientes parámetros:

#@markdown *   **DF_NAME**: Nombre del conjunto de datos con el que se desea comparar el actual. Debe estar en la carpeta WS_PATH y debe de ser un fichero de  formato csv. **(Ejemplo: booksummaries.csv)**.

#@markdown *   **EMBD_COLUMN_COMPARE**: Nombre de la columna del conjunto de datos que contiene los textos que se compararan con el conjunto actual.

#@markdown *   **NUM_SAVE** : Número de elementos similares que se desea obtener.

#@markdown *   **SAVE_SIMILARITY_MATRIX_PICKLE**: Si la casilla está marcada la matriz de similitudes será guardada en formato .pkl con el nombre indicado en OUTPUT_NAME. Si se quieren cargar posteriormente en este cuaderno se deben de guardar con este formato. 

#@markdown *   **SAVE_SIMILARITY_MATRIX_CSV**: Si la casilla está marcada la matriz de similitudes será guardada en formato .csv con el nombre indicado en OUTPUT_NAME. Usar cuando se quiera exportar los datos a otra plataforma en la que utilizar el archivo en formato csv.

#@markdown *   **OUTPUT_NAME** :  Nombre con el que serán guardada la matriz de similitudes. **(Ejemplo: similitudes_elementos)**



#@markdown <br>
DF_NAME=  "arxiv_data.csv" #@param {type:"string"}
EMBD_COLUMN_COMPARE=  "titles" #@param {type:"string"}
NUM_SAVE = 25 #@param {type:"integer"}
SAVE_SIMILARITY_MATRIX_PICKLE = True #@param {type:"boolean"}
SAVE_SIMILARITY_MATRIX_CSV = False #@param {type:"boolean"}
OUTPUT_NAME ="similarity with arxiv titles" #@param {type:"string"}


df_compare = pd.read_csv(WS_PATH + DF_NAME)
if EMBD_COLUMN_COMPARE not in df_compare.columns:
  print("Columnas cargadas: ", list(df_compare.columns))
  raise Exception("La columna {} no se encuentra en el conjunto de datos cargado.".format(EMBD_COLUMN_COMPARE))

df_compare[EMBD_COLUMN_COMPARE] = df_compare[EMBD_COLUMN_COMPARE].astype("string")
print("Número de elementos cargados: ", len(df_compare), '\n')

print('Usando el modelo:\n')
print(BIENCODER_MODEL)

print('\nComputing query embeddings...')
query_embeddings = BIENCODER_MODEL.encode(df_compare[EMBD_COLUMN_COMPARE],
                                    convert_to_tensor=False,
                                    device = str(device),
                                    batch_size = 32,
                                    show_progress_bar=True,
                                    normalize_embeddings= True)


def fun_cosenos_df_compare(NUM_SAVE: int):
 #Configuramos faiss
  if torch.cuda.is_available():
    cfg = faiss.GpuIndexFlatConfig()
    cfg.useFloat16 = True
    index = faiss.GpuIndexFlatIP(faiss.StandardGpuResources(), embeddings.shape[1], cfg)
  else:
    index = faiss.IndexFlatIP(embeddings.shape[1])
    
  index.add(embeddings)

  D, I = index.search(query_embeddings, NUM_SAVE)
  return list(D), list(I)

if NUM_SAVE >= len(df):
  NUM_SAVE = len(df)  - 1
if NUM_SAVE > 2048:
  NUM_SAVE = 2048

cosenos_df_compare, index_df_compare = fun_cosenos_df_compare(NUM_SAVE)
df_res_compare = pd.DataFrame()
df_res_compare['index_compare'] = index_df_compare
df_res_compare['cosenos_compare'] = cosenos_df_compare

if SAVE_SIMILARITY_MATRIX_CSV:
  print("Guardando {}.csv...".format(OUTPUT_NAME))
  df_res_compare.to_csv(similarity_matrix_dir + OUTPUT_NAME + ".csv",index=False)
if SAVE_SIMILARITY_MATRIX_PICKLE:
  print("Guardando {}.pkl...".format(OUTPUT_NAME))
  df_res_compare.to_pickle(similarity_matrix_dir + OUTPUT_NAME + ".pkl",protocol = pickle.HIGHEST_PROTOCOL)

print("Mostando primeros elementos...\n")
data_table.DataTable(df_res_compare.head(10), include_index=True, num_rows_per_page=5)